<center><h1>Big Data Bowl 2022</h1></center>
<center><h2>Evaluating Headhunters: KIPPER</h2></center>
<center><h3>Kickoff and Punt Player Efficiency Rating</h3></center>

Special teams units are filled with the unsung heroes of the game. The irony is that greatness on special teams is excellent job security because great teams have great special teams units. Football is a game of inches and champions value every inch of better field position. I am creating a metric to capture the true value of performance of these headhunters on the field.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.options.mode.chained_assignment = None


import os

players_df = pd.read_csv('../input/nfl-big-data-bowl-2022/players.csv')
pff_data_df = pd.read_csv('../input/nfl-big-data-bowl-2022/PFFScoutingData.csv')
tracking2020_df = pd.read_csv('../input/nfl-big-data-bowl-2022/tracking2020.csv')
games_df = pd.read_csv('../input/nfl-big-data-bowl-2022/games.csv')
plays_df = pd.read_csv('../input/nfl-big-data-bowl-2022/plays.csv')


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 <center>Combine Plays Dataframe and Tracking 2020 on playID</center>

In [ ]:
plays_tracking2020_df = pd.merge(plays_df, tracking2020_df, left_on=['playId', plays_df['specialTeamsPlayType'].str.lower()], right_on=['playId', 'event'])

In [ ]:
plays_tracking2020_df = plays_tracking2020_df.rename(columns={"gameId_y": "gameId"})

# <center>Max Acceleration By Player 2020</center>

In [ ]:
max_a_df = plays_tracking2020_df.loc[plays_tracking2020_df.groupby("nflId")['a'].idxmax()]
player_2020_master_df = max_a_df[['nflId', 'a', 'displayName', 'position']].copy()
player_2020_master_df = player_2020_master_df.rename(columns={"a": "max_acceleration"})
player_2020_master_df = player_2020_master_df.reset_index(drop=True)
player_2020_master_df.head(10)

 <center>Organize players by snaps played on Kickoff and Punt in 2020</center>

In [ ]:
unique_plays_tracking2020_df = plays_tracking2020_df.drop_duplicates(subset=['playId'])
unique_kickoffs_tracking2020_df = unique_plays_tracking2020_df[unique_plays_tracking2020_df['specialTeamsPlayType'] == 'Kickoff']
unique_kickoffs_tracking2020_df['nflId'].value_counts()

player_2020_kickoff_aggregate = {}

for nfl_Id in player_2020_master_df.nflId:
    kickoff_snaps = unique_kickoffs_tracking2020_df.loc[(unique_kickoffs_tracking2020_df['nflId'] == nfl_Id)].shape[0]
    
    if kickoff_snaps > 0:
        player_2020_kickoff_aggregate[nfl_Id] = kickoff_snaps
    else:
        player_2020_kickoff_aggregate[nfl_Id] = 0

player_2020_kickoff_snaps_series = pd.Series(player_2020_kickoff_aggregate)
player_2020_master_df['kickoff_snaps'] = player_2020_kickoff_snaps_series.values


unique_plays_tracking2020_df = plays_tracking2020_df.drop_duplicates(subset=['playId'])
unique_punts_tracking2020_df = unique_plays_tracking2020_df[unique_plays_tracking2020_df['specialTeamsPlayType'] == 'Punt']
unique_punts_tracking2020_df.head(20)
unique_punts_tracking2020_df['nflId'].value_counts()

player_2020_punt_aggregate = {}

for nfl_Id in player_2020_master_df.nflId:
    punt_snaps = unique_punts_tracking2020_df.loc[(unique_punts_tracking2020_df['nflId'] == nfl_Id)].shape[0]
    
    if punt_snaps > 0:
        player_2020_punt_aggregate[nfl_Id] = punt_snaps
    else:
        player_2020_punt_aggregate[nfl_Id] = 0
        
player_2020_punt_snaps_series = pd.Series(player_2020_punt_aggregate)
player_2020_master_df['punt_snaps'] = player_2020_punt_snaps_series.values
player_2020_master_df['punt_or_kickoff'] = player_2020_master_df['kickoff_snaps'] + player_2020_master_df['punt_snaps']

 <center>Get the team and jersey for each player (Including Team Switches)</center>

In [ ]:
unique_plays_tracking2020_df = plays_tracking2020_df.drop_duplicates(subset=['playId'])
games_plays_tracking2020_df = pd.merge(plays_tracking2020_df, games_df[['gameId','homeTeamAbbr','visitorTeamAbbr']], on=['gameId'])
games_plays_tracking2020_df = games_plays_tracking2020_df.sort_values(by=['nflId'], ascending=True)

player_2020_team_aggregate = {}
for i in games_plays_tracking2020_df.index:
    
    nflId = games_plays_tracking2020_df['nflId'][i]
    displayName = games_plays_tracking2020_df['displayName'][i]
    team = games_plays_tracking2020_df['team'][i]
    homeTeamAbbr = games_plays_tracking2020_df['homeTeamAbbr'][i]
    visitorTeamAbbr = games_plays_tracking2020_df['visitorTeamAbbr'][i]
    jerseyNumber = games_plays_tracking2020_df['jerseyNumber'][i]
    
    if team == 'home':
        
        jerseyNumber = str(int(jerseyNumber))
        try:
            player_2020_team_aggregate[f'{nflId}'].add(f'{homeTeamAbbr} {jerseyNumber}')
        except KeyError:
            player_2020_team_aggregate[f'{nflId}'] = {f'{homeTeamAbbr} {jerseyNumber}'}
    elif team == 'away':
        
        jerseyNumber = str(int(jerseyNumber))
        try:
            player_2020_team_aggregate[f'{nflId}'].add(f'{visitorTeamAbbr} {jerseyNumber}')
        except KeyError:
            player_2020_team_aggregate[f'{nflId}'] = {f'{visitorTeamAbbr} {jerseyNumber}'}

player_2020_team_series = pd.Series(player_2020_team_aggregate)
player_2020_master_df['team_jerseyNumber'] = player_2020_team_series.values
player_2020_master_df.head(20)

<center>Get PFF Dataframe for the 2020 season</center>

In [ ]:
games_pff_df = pd.merge(pff_data_df, games_df, left_on=['gameId'], right_on=['gameId'])
pff_2020_df = games_pff_df[(games_pff_df['season'] == 2020)]

 <center><h1>Organize Players By Tackles Made and Missed</h1></center>


In [ ]:
solo_tackle_aggregate = {}

for nflId in player_2020_master_df['nflId']:
    
    team_jerseyNumbers = player_2020_master_df.loc[(player_2020_master_df['nflId'] == nflId)].team_jerseyNumber.iloc[0]
    
    total_solo_tackles = 0
    for team_jerseyNumber in team_jerseyNumbers:
        
        team_jerseyNumber = team_jerseyNumber.strip()
        total_solo_tackles += pff_2020_df.loc[(pff_2020_df['tackler'] == team_jerseyNumber)].shape[0] 
    
    nflId = str(int(nflId))
    solo_tackle_aggregate[f'{nflId}'] = total_solo_tackles

solo_tackle_aggregate_series = pd.Series(solo_tackle_aggregate)
player_2020_master_df['solo_tackles'] = solo_tackle_aggregate_series.values

<center>Organize Players By Assisted Tackles Made</center>

In [ ]:
player_2020_assist_tackle_aggregate = {}

for nflId in player_2020_master_df['nflId']:
    
    team_jerseyNumbers = player_2020_master_df.loc[(player_2020_master_df['nflId'] == nflId)].team_jerseyNumber.iloc[0]
    
    total_assist_tackles = 0
    for team_jerseyNumber in team_jerseyNumbers:
        
        team_jerseyNumber = team_jerseyNumber.strip()
        total_assist_tackles += pff_2020_df.loc[(pff_2020_df['assistTackler'] == team_jerseyNumber)].shape[0] 
    
    nflId = str(int(nflId))
    player_2020_assist_tackle_aggregate[f'{nflId}'] = total_assist_tackles
    
player_2020_assist_tackle_series = pd.Series(player_2020_assist_tackle_aggregate)
player_2020_master_df['assist_tackles'] = player_2020_assist_tackle_series.values

 <center>Get Total Tackles</center>

In [ ]:
player_2020_master_df['total_tackles'] = player_2020_master_df['solo_tackles'] + (player_2020_master_df['assist_tackles'] / 2)

 <center>Organize Players by Missed Tackles</center>

In [ ]:
player_2020_missed_tackle_aggregate = {}

for nflId in player_2020_master_df['nflId']:
    
    team_jerseyNumbers = player_2020_master_df.loc[(player_2020_master_df['nflId'] == nflId)].team_jerseyNumber.iloc[0]
    total_missed_tackles = 0
    for team_jerseyNumber in team_jerseyNumbers:

        team_jerseyNumber = team_jerseyNumber.strip()
        total_missed_tackles += pff_2020_df['missedTackler'].str.contains(team_jerseyNumber).sum() 
    
    nflId = str(int(nflId))
    player_2020_missed_tackle_aggregate[f'{nflId}'] = total_missed_tackles
    
player_2020_missed_tackle_series = pd.Series(player_2020_missed_tackle_aggregate)
player_2020_master_df['missed_tackles'] = player_2020_missed_tackle_series.values

 <center>Get Net Tackles</center>

In [ ]:
player_2020_master_df['net_tackles'] = player_2020_master_df['total_tackles'] - player_2020_master_df['missed_tackles']

 <center>Get Tackling Efficiency for each player</center>

In [ ]:
player_2020_master_df['tackle_efficiency'] = player_2020_master_df['net_tackles'] / player_2020_master_df['punt_or_kickoff']

 <center>Players with more than 30 Kickoff or Punt snaps in 2020</center>

In [ ]:
filtered_master_dataframe = player_2020_master_df.loc[(player_2020_master_df['punt_or_kickoff'] > 30)]

# <center>Correlation Between Acceleration and Net Tackles</center>

Max acceleration has a 77.57% positive correlation with net tackles, while max speed had a week positive correlation. This makes perfect sense because anyone who played knows that straightforward speed is not enough, you have to be able to change direction quickly and meet the target at an angle. Or as an old coach said "You meet them where they are going to be, not where they are". Usama Young also referenced this in the starter video when he said:
“Speed and angles, you can’t teach speed but oftentimes angles beat speed”. 

And when it comes to fighting off blocks: **ACCELERATION CREATES SEPERATION!**

In [ ]:
max_acceleration = filtered_master_dataframe["max_acceleration"]
total_tackles = filtered_master_dataframe["net_tackles"]
max_acceleration_tackles_correlation = max_acceleration.corr(total_tackles)
print(max_acceleration_tackles_correlation)

In [ ]:
max_acceleration_tackles_line = filtered_master_dataframe.plot.line(x='max_acceleration', y='total_tackles')

# <center>Create ranking of players by kickoff and punt snaps</center>

The KIPPER rating (Please don't sue me John Hollinger) factors in the player's net tackling efficiency and their acceleration. Acceleration impacts more than just tackling, the change in direction is also needed to properly fill a lane on kickoff and punt. 

In [ ]:
filtered_master_dataframe['efficiency_rank'] = filtered_master_dataframe['tackle_efficiency'].rank(ascending=False)
filtered_master_dataframe['max_acceleration_rank'] = filtered_master_dataframe['max_acceleration'].rank(ascending=False)

In [ ]:
filtered_master_dataframe['kipper'] = filtered_master_dataframe['efficiency_rank'] + filtered_master_dataframe['max_acceleration_rank']

In [ ]:
filtered_master_dataframe = filtered_master_dataframe.sort_values('kipper')
rank_columns = ['nflId','displayName', 'kipper', 'efficiency_rank', 'max_acceleration_rank', 'tackle_efficiency', 'team_jerseyNumber']
rank_master_dataframe_30_snaps = pd.DataFrame(filtered_master_dataframe, columns=rank_columns)
rank_master_dataframe_30_snaps.head(10)

 <center><h1>Create a rank of players with 20 or more snaps</h1></center>

In [ ]:
filtered_master_dataframe_20 = player_2020_master_df.loc[(player_2020_master_df['punt_or_kickoff'] > 20)]
max_acceleration_20 = filtered_master_dataframe_20["max_acceleration"]
net_tackles_20 = filtered_master_dataframe_20["net_tackles"]

filtered_master_dataframe_20['efficiency_rank'] = filtered_master_dataframe_20['tackle_efficiency'].rank(ascending=False)
filtered_master_dataframe_20['max_acceleration_rank'] = filtered_master_dataframe_20['max_acceleration'].rank(ascending=False)

filtered_master_dataframe_20['kipper'] = filtered_master_dataframe_20['efficiency_rank'] + filtered_master_dataframe_20['max_acceleration_rank']

filtered_master_dataframe_20 = filtered_master_dataframe_20.sort_values('kipper')
rank_master_dataframe_20_snaps = pd.DataFrame(filtered_master_dataframe_20, columns=rank_columns)
rank_master_dataframe_20_snaps.head(10)

<center><h1>Create a rank of players with 10 or more snaps</h1></center>

In [ ]:
filtered_master_dataframe_10 = player_2020_master_df.loc[(player_2020_master_df['punt_or_kickoff'] > 10)]
max_acceleration_10 = filtered_master_dataframe_10["max_acceleration"]
net_tackles_10 = filtered_master_dataframe_10["net_tackles"]

filtered_master_dataframe_10['efficiency_rank'] = filtered_master_dataframe_10['tackle_efficiency'].rank(ascending=False)
filtered_master_dataframe_10['max_acceleration_rank'] = filtered_master_dataframe_10['max_acceleration'].rank(ascending=False)

filtered_master_dataframe_10['kipper'] = filtered_master_dataframe_10['efficiency_rank'] + filtered_master_dataframe_10['max_acceleration_rank']

filtered_master_dataframe_10 = filtered_master_dataframe_10.sort_values('kipper')
rank_master_dataframe_10_snaps = pd.DataFrame(filtered_master_dataframe_10, columns=rank_columns)


In [ ]:
rank_master_dataframe_10_snaps.head(10)

In [ ]:
player_2020_master_df.to_csv('player_2020_master_df.csv')
filtered_master_dataframe.to_csv('filtered_master_dataframe.csv')

# <center>Conclusion</center>

A clearly defined metric makes special teams personnel management much more straightforward. A KIPPER rank can be calculated every 10 kickof + punt snaps, roughly equating to each game. This gives so much opportunity to players in the preseason and in training camp. This does not even factor the many ways I am improving KIPPER after the current round of the competition, like accomodating kickers and punters.

For players over 30 snaps, the fact that the top 3 (**Jordan Dangerfield**, **Sherrick McManis**, and **Craig Robertson**) were all special teams captains last season whie the 4th (**Anthony Levine**) was a co-captain shows that KIPPER is looking at the right features and can only get more refined. Jordan Dangerfield having the best KIPPER on 10+ snaps, 20+ snaps, and 30+ snaps also shows how elite and consistent he was.


# <center>Bonus Content</center>
<center>Found a video giving credit to our MVP last season!</center>

<iframe width="1250" height="703" src="https://www.youtube.com/embed/lVipB3i3H1g" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>